# HCR Pre-Processing


## *** Author: Keshava Prasad Gubbi***
## Script for pre-processing HCR Lines.
## Read the filenames and split and determine names of sig1, sig2 and ref channels
## Split the Czi imagefilename into respective channels and save it as nrrd with respective names as per channels.

In [1]:
import os
import numpy as np
from aicspylibczi import CziFile
import re
import nrrd
from aicsimageio import AICSImage

In [2]:
def create_channel_folder(path):
    if not os.path.exists(path):
        print(f'Creating {path}')
        os.makedirs(path, exist_ok=True)
        return path

### ########Splitting Channels####################

### Filetype B (with ch1 as reference Channel) - For the new style of naming images!

### Please enter the path (full folder path into the file_path variable) for the line to be pre-processed!

In [3]:
file_path = r'C:\Users\keshavgubbi\Desktop\HCR\raw_data\20210302_rspo1_cckb'
preprocessed_path = file_path + '/preprocessed/'
# ref_num = int(input('Enter Reference Channel Number: (please enter 0 if refrence channel in image is channel1!)'))
ref_num = 0

In [4]:
########Splitting Channels####################

for file in os.listdir(file_path):
    if file.endswith('.czi'):
        print('Image name: ', file)
        create_channel_folder(preprocessed_path)

        e_name, sig_ch1_name, sig_ch2_name, ref_ch_name = re.split(r'_ch\d_', file)
        c, ext = ref_ch_name.split('.', 1)
        ref_ch_name, fish_num = c.split('_', 1)
        e1_name, f = e_name.rsplit('_', 1)
        days, embryo_name = e1_name.split('_', 1)

        print('fish_num:', fish_num)
        print('ref_ch_name:', ref_ch_name)
        print('sig_ch1_name:', sig_ch1_name)
        print('sig_ch2_name:', sig_ch2_name)
        print('embryo_name:', embryo_name)

        c = AICSImage(os.path.join(file_path, file))

        # Obtain the image data from respective channels
        first_channel_data = c.get_image_data("ZYX", C=0, S=0, T=0)
        second_channel_data = c.get_image_data("ZYX", C=1, S=0, T=0)
        third_channel_data = c.get_image_data("ZYX", C=2, S=0, T=0)

        RImage = np.stack(first_channel_data).astype('uint8')
        S1Image = np.stack(second_channel_data).astype('uint8')
        S2Image = np.stack(third_channel_data).astype('uint8')

        ### Writing the individual Channels into nrrd format
        print('Writing the individual Channels into nrrd format!')

        print(f'Creating Reference Channel nrrd file with name : {embryo_name}_{fish_num}_ch0_{ref_ch_name}.nrrd ')
        nrrd.write(os.path.join(preprocessed_path, f"{embryo_name}_{fish_num}_ch0_{ref_ch_name}.nrrd"), RImage, index_order='C')
        print(f'Creating nrrd file for Signal1 Channel with name : {embryo_name}_{fish_num}_ch1_{sig_ch1_name}.nrrd')
        nrrd.write(os.path.join(preprocessed_path, f"{embryo_name}_{fish_num}_ch1_{sig_ch1_name}.nrrd"), S1Image, 
                   index_order='C')
        print(f'Creating nrrd file Signal2 Channel with name : {embryo_name}_{fish_num}_ch2_{sig_ch2_name}.nrrd')
        nrrd.write(os.path.join(preprocessed_path, f"{embryo_name}_{fish_num}_ch2_{sig_ch2_name}.nrrd"), S2Image,
                   index_order='C')

        print(f'###################### Completed processing {file} ###################### ')

Image name:  6dpf_huc_h2b_gcamp6s_ch3_cckb_ch2_rspo1_ch1_GC6s_1.czi
fish_num: 1
ref_ch_name: GC6s
sig_ch1_name: cckb
sig_ch2_name: rspo1
embryo_name: huc_h2b


C:\Users\keshavgubbi\Anaconda3\lib\site-packages\aicsimageio\transforms.py:177: UserWarning: Data has dimension B with depth 1, assuming B=0 is the desired value, if not the case specify B=x where x is an integer, list, tuple, range, or slice.
  warnings.warn(


Writing the individual Channels into nrrd format!
Creating Reference Channel nrrd file with name : huc_h2b_1_ch0_GC6s.nrrd 
Creating nrrd file for Signal1 Channel with name : huc_h2b_1_ch1_cckb.nrrd
Creating nrrd file Signal2 Channel with name : huc_h2b_1_ch2_cckb.nrrd
###################### Completed processing 6dpf_huc_h2b_gcamp6s_ch3_cckb_ch2_rspo1_ch1_GC6s_1.czi ###################### 
Image name:  6dpf_huc_h2b_gcamp6s_ch3_cckb_ch2_rspo1_ch1_GC6s_2.czi
fish_num: 2
ref_ch_name: GC6s
sig_ch1_name: cckb
sig_ch2_name: rspo1
embryo_name: huc_h2b
Writing the individual Channels into nrrd format!
Creating Reference Channel nrrd file with name : huc_h2b_2_ch0_GC6s.nrrd 
Creating nrrd file for Signal1 Channel with name : huc_h2b_2_ch1_cckb.nrrd
Creating nrrd file Signal2 Channel with name : huc_h2b_2_ch2_cckb.nrrd
###################### Completed processing 6dpf_huc_h2b_gcamp6s_ch3_cckb_ch2_rspo1_ch1_GC6s_2.czi ###################### 
Image name:  6dpf_huc_h2b_gcamp6s_ch3_cckb_ch2_rspo1_ch1

# Creating script for cluster

In [14]:
#root_dir = r'C:\Users\keshavgubbi\Desktop\HCR'
root_dir = r"C:\Users\keshavgubbi\Desktop\HCR"
output_dir = r"C:\Users\keshavgubbi\Desktop\HCR\AutomatedScripts"
template_file = r"C:\Users\keshavgubbi\Desktop\HCR\20210615_HCR_alignment_1_1st_new.sh"

In [13]:
# os.listdir(root_dir) gaves me all the fish folders names inside root_dir
for folder_name in os.listdir(root_dir):
    #print(folder_name)
    if not os.path.isfile(os.path.join(root_dir, f'HCR_alignment_{embryo_name}.sh')):
        # path of scripts files
        alignment_script_file = os.path.join(output_dir, f'HCR_alignment_{embryo_name}.sh')
        
#         # create new alignment script file
#         alignment_script_file = open(output_script_path, "wt+")
#         print(f"The averaging script for {embryo_name} has been created!")
        
#         # open template file
#         template_file = open(avg_template_path, 'rt')
        
        with open(template_file, 'r') as templatefile, open(alignment_script_file, 'a') as scriptfile:
            for line in templatefile:
                print(line)
                scriptfile.write(line)
        
#         # copy the template file to the new script file
#         for line in template_file:
#             print(line)
#             if line.strip() == "HCR=pyya_pyyb_1":
#                 alignment_script_file.write("HCR=" + embryo_name.replace('\r', ''))
#             if line.strip() == "ch2=pyyb":
#                 alignment_script_file.write("ch2=" + sig_ch1_name.replace('\r', ''))
#             if line.strip() == "ch3=pyya":
#                 alignment_script_file.write("ch2=" + sig_ch2_name.replace('\r', ''))

#         print(f"The averaging script for {embryo_name} has been created!")